# Demo of `MotionList`

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import xarray as xr

In [ ]:
try:
    from bapsf_motion.motion_list import MotionList
except ModuleNotFoundError:
    from pathlib import Path

    HERE = Path().cwd()
    BAPSF_MOTION = (HERE / ".." / ".." / ".." / "src").resolve()
    sys.path.append(str(BAPSF_MOTION))
    
    from bapsf_motion.motion_list import MotionList

In [ ]:
plt.rcParams.update(
    {
        # "figure.figsize": [12, 0.56 * 12],
        "figure.figsize": [10, 0.8 * 10],
        "font.size": 16,
    }
)

Let's set up a typical rectangular grid for a probe on the East port.

In [ ]:
ml = MotionList(
    space=[
        {"label": "x", "range": [-55, 55], "num": 221},
        {"label": "y", "range": [-55, 55], "num": 221},
    ],
    exclusions=[
        {"type": "lapd-XY", "port_location": "E", "cone_full_angle": 60},
    ],
    layers=[
        {"type": "grid", "limits": [[0, 30], [-30, 30]], "steps": [11, 21]},
    ],
)

ml.mask.plot(x="x", y="y");

points = ml._ds["point_layer1"].data
flat_ax = np.prod(points.shape[:-1])
points = np.reshape(points, (flat_ax, points.shape[-1]))
pt1 = points
# print(pt1.shape)

plt.scatter(
    points[...,0],
    points[..., 1], s=4**2,
    color="red",
)

valid_points = ml.motion_list

plt.scatter(
    valid_points[..., 0],
    valid_points[..., 1],
    linewidth=1,
    s=6**2,
    facecolors="deepskyblue",
    edgecolors="black",
);

In [ ]:
ml.motion_list.shape

In [ ]:
ml.config

Let's say we want the same setup as before, but with a probe on the 45 degree port.  This is easily accomplished by just specifying the `port_location` as an angle for the LaPD exclusion layer.

In [ ]:
ml = MotionList(
    space="lapd_xy",
    exclusions=[
        {"type": "lapd-XY", "port_location": 45, "cone_full_angle": 60},
    ],
    layers=[
        {"type": "grid", "limits": [[0, 30], [-30, 30]], "steps": [11, 21]},
    ],
)

ml.mask.plot(x="x", y="y");

points = ml._ds["point_layer1"].data
flat_ax = np.prod(points.shape[:-1])
points = np.reshape(points, (flat_ax, points.shape[-1]))
pt1 = points
# print(pt1.shape)

plt.scatter(
    points[...,0],
    points[..., 1], s=4**2,
    color="red",
)

valid_points = ml.motion_list

plt.scatter(
    valid_points[..., 0],
    valid_points[..., 1],
    linewidth=1,
    s=6**2,
    facecolors="deepskyblue",
    edgecolors="black",
);

Let's se we just want a vertical line for a target in the top port.

In [ ]:
ml = MotionList(
    space="lapd_xy",
    exclusions=[
        {"type": "lapd-XY", "port_location": "T", "cone_full_angle": 60},
    ],
    layers=[
        {"type": "grid", "limits": [[0, 0], [-20, 20]], "steps": [1, 11]},
    ],
)

ml.mask.plot(x="x", y="y");

points = ml._ds["point_layer1"].data
flat_ax = np.prod(points.shape[:-1])
points = np.reshape(points, (flat_ax, points.shape[-1]))
pt1 = points
# print(pt1.shape)

plt.scatter(
    points[...,0],
    points[..., 1], s=4**2,
    color="red",
)

valid_points = ml.motion_list

plt.scatter(
    valid_points[..., 0],
    valid_points[..., 1],
    linewidth=1,
    s=6**2,
    facecolors="deepskyblue",
    edgecolors="black",
);

Let's say we have a probe onthe West port and want to take two grids of data where the first grid has a low density of points over a large area and the seconde gride has a high density of points over small area.  This is easily accomplished by just defining the two grids as separate layers.  The order in with the points will be taken depends on the order in which the layers are specified.  In this example, the low density grid is completely measured first and then followed by the high density grid.

In [ ]:
ml = MotionList(
    space="lapd_xy",
    exclusions=[
        {"type": "lapd-XY", "port_location": "W", "cone_full_angle": 60},
    ],
    layers=[
        {"type": "grid", "limits": [[-30, 30], [-20, 20]], "steps": [21, 11]},
        {"type": "grid", "limits": [[-20, 0], [-10, 10]], "steps": [15, 11]},
    ],
)

ml.mask.plot(x="x", y="y");

for player in ml.layers:
    points = player.points.data
    flat_ax = np.prod(points.shape[:-1])
    points = np.reshape(points, (flat_ax, points.shape[-1]))

    plt.scatter(
        points[...,0],
        points[..., 1], s=4**2,
        color="red",
    )

valid_points = ml.motion_list

plt.scatter(
    valid_points[..., 0],
    valid_points[..., 1],
    linewidth=1,
    s=6**2,
    facecolors="deepskyblue",
    edgecolors="black",
);

In [ ]:
ml.config

In [ ]:
import tomli_w
from pprint import pprint

In [ ]:
cstr = tomli_w.dumps(ml.config, multiline_strings=False)
pprint(cstr)

In [ ]:
for key, val in ml.config["exclusion"]["0"].items():
    print(key, val, type(val))

In [ ]:
isinstance(ml.config["exclusion"]["0"]["pivot_radius"], np.generic)
